## Librerías

In [9]:
# Importación de paquetes necesarios en caso de no tenerlos

%%capture
!pip install tiktoken
!pip install openai

In [1]:
# Librerías necesarias
import tiktoken as tiktoken
import json
import requests
from math import ceil
import openai
openai.api_key = "agregar key de OpenAI"

## Pruebas con GPT

In [7]:
# Chat a analizar
import pandas as pd
id_user = 3200

data = pd.read_csv("clean_julieta.csv")
Ind1 = data[data['user_id'] == id_user][['opt_left', 'opt_right', 'sel', 'comment', 'etapa']].iloc[0]
Grup = data[data['user_id'] == id_user][['opt_left', 'opt_right', 'sel', 'comment', 'etapa']].iloc[1]
Ind2 = data[data['user_id'] == id_user][['opt_left', 'opt_right', 'sel', 'comment', 'etapa']].iloc[2]

text_file = open("chat_julieta/team_1546.0_chat.txt", "r")
data = text_file.read()
text_file.close()

prompt = f"""Se realizó un estudio en un grupo de estudiantes en el que se les presentó un dilema ético 
y las dos opciones entre las que pueden decidir. Los estudiantes participaron de 3 instancias: una individual, 
una grupal y finalmente otra individual. El objetivo de los estudiantes es manifestar su postura entre ambas 
opciones usando valores entre 1 y 6. Se te proporciona 
la información de cada una de las etapas en las que participaron los estudiantes, es decir, en la primera etapa 
individual tendrás la valorización y justificación del estudiante, en la etapa grupal tendrás un chat de grupo 
en el que los estudiantes discuten y llegan a un acuerdo de valorización en conjunto junto con su justificación, 
y finalmente en la última etapa individual vuelves a tener la valorización y justificación del estudiante tras 
haber participado de la instancia grupal. Tus tareas son: primero, identificar en las justificaciones textuales los 
elementos principales que sostienen la postura (de 1 a 6) del estudiante en cada instancia individual;
luego identificar los elementos que sostienen la postura grupal usando los registros del chat, después identificar 
aquellos elementos que cambiaron en las justificaciones individuales entre la primera y segunda instancia,
identificando qué elementos del chat grupal están relacionados con la justificación de la segunda instancia.
Todo esto debe ser entregado en una respuesta breve, no más de 100 palabras.
La valorización y justificación de la instancia individual 1 son: {Ind1}
La valorización y justificación de la instancia grupal son: {Grup}
La valorización y justificación de la instancia individual 2 son: {Ind2}
El chat grupal se proporciona a continuación: {data}"""

# Input para la API
messages = [
    {"role": "user",
     "name": "user",
     "content": prompt}
]

# Modelo a usar
model = "gpt-3.5-turbo-16k-0613"

# Output tokens
max_tokens=1000

# Respuesta de la API
response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0.0,
            max_tokens=max_tokens,
        )

# Output de la API
output_tokens = response.usage.completion_tokens
output = response.choices[0].message.content
output

'En la primera instancia individual, el estudiante valora la opción de no usar la información del grupo de WhatsApp con un 6, argumentando que afectaría su futuro. En la instancia grupal, el estudiante valora la misma opción con un 4, argumentando que es un punto medio entre copiar y no copiar, y que quizás copiar es la única opción que le queda si su beca está en riesgo. En la segunda instancia individual, el estudiante valora la opción con un 5, argumentando que su posición cambió un poco y que sigue creyendo que copiar es una opción si su beca está en riesgo. En el chat grupal, se discute sobre las ventajas y desventajas de copiar, la importancia de la beca y las consecuencias de copiar.'

## Estimación de precios

In [8]:
# Función oficial de OpeanAI para obtener el número de tokens

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [9]:
# API gratuita para obtener el precio actualizado del dólar a pesos chilenos

class Mindicador:
    def __init__(self, indicador):
        self.indicador = indicador


    def InfoApi(self):
        # En este caso hacemos la solicitud para el caso de consulta de un indicador en un año determinado
        url = f'https://mindicador.cl/api/{self.indicador}'
        response = requests.get(url)
        data = json.loads(response.text.encode("utf-8"))
        # Para que el json se vea ordenado, retornar pretty_json
        pretty_json = json.dumps(data, indent=2)
        return data

In [10]:
# Función para obtener el precio total de un prompt

def tokens(text, model, output_tokens):
    messages = [
            {"role": "user",
             "name": "user",
             "content": text
             }
        ]

    # Number of tokens
    n_tokens = num_tokens_from_messages(messages, model)

    # Dollar to chilean pesos
    indicador = Mindicador("dolar")
    dolar = indicador.InfoApi()["serie"][0]["valor"]

    # Dictionaries with respective prices
    input_dict = {"gpt-3.5-turbo-0613": 0.0015, "gpt-3.5-turbo-16k-0613": 0.003, "gpt-4-0314": 0.03, "gpt-4-32k-0314":
        0.06, "gpt-4-0613": 0.03, "gpt-4-32k-0613": 0.06}
    output_dict = {"gpt-3.5-turbo-0613": 0.002, "gpt-3.5-turbo-16k-0613": 0.004, "gpt-4-0314": 0.06, "gpt-4-32k-0314":
        0.12, "gpt-4-0613": 0.06, "gpt-4-32k-0613": 0.12}
    
    input_price = ceil(input_dict[model] * n_tokens // 1000)
    input_precio = ceil(input_dict[model] * dolar * n_tokens // 1000)
    output_price = ceil(output_dict[model] * output_tokens // 1000)
    output_precio = ceil(output_dict[model] * dolar * output_tokens // 1000)
    resp_dic = {"model": model,
                "numTokens": n_tokens,
                "input price [USD]": input_price,
                "output price [USD]": output_price,
                "total price [USD]": input_price+output_price,
                "input precio [CLP]": input_precio,
                "output precio [CLP]": output_precio,
                "total precio [CLP]": input_precio+output_precio}
    return resp_dic

In [11]:
tokens(prompt, model, output_tokens)

{'model': 'gpt-3.5-turbo-16k-0613',
 'numTokens': 3276,
 'input price [USD]': 0,
 'output price [USD]': 0,
 'total price [USD]': 0,
 'input precio [CLP]': 8,
 'output precio [CLP]': 0,
 'total precio [CLP]': 8}